# Ejercicio práctico - Tema 3: Modelo de Lenguaje

**Alumno**: Iván Cañaveral Sánchez

## Modelo de lenguaje

Para dar respuesta a lso distintos apartados de este ejercicio práctico vamos a definir una clase cuyos métodos irán utilizándose en los distintos apartados.

Esta clase toma como parámetros:

* El corpus de entrenamiento
* Una variable binaria que indica si se aplica o no Laplace Smoothing.

Los métodos más relevantes que implementa son:

* **word_prob**: calcula la probabilidad condicionada de la palabra `w_n`, dado que aparece `w_(n-1)`.
* **next_most_suitable_word**: calcula la siguiente palabra más probable dada una secuencia.
* **sentence_prob**: calcula la probabilidad de una frase.
* **amount_of_information**: calcula la cantidad de información de una frase.
* **perplexity**: calcula la perplejidad de una frase.

In [ ]:
import re
import numpy as np
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer
from numpy.lib.stride_tricks import sliding_window_view

In [ ]:
class BigramModel():
  """
  Class for languaje models based on bigrams.
  A n-gram model can be developed using this class,
  just changing previous_words by a list or string
  of previous words.
  """
  def __init__(self, corpus, laplace_smoothing=False):
    self.corpus = corpus
    self.laplace_smoothing = laplace_smoothing
    self.tokenizer = lambda text: re.split("\\s+", text)
    self.text = ' '.join(corpus)
    self.words = list(
        set(self.tokenizer(self.text.lower()))
    )
    self.vocab_size = len(self.words)
    self.bigrams = list(
        set(
            [' '.join(pair) for pair in product(self.words, self.words) \
             if ' '.join(pair) in self.text.lower()]
            )
        )
    self.bigram_cv = CountVectorizer(
        ngram_range=(2,2),
        vocabulary=self.bigrams,
        tokenizer=self.tokenizer
    )
    self.word_cv = CountVectorizer(
        vocabulary=self.words,
        tokenizer=self.tokenizer
    )
    self.word_count = self._get_word_count()
    self.bigram_count = self._get_bigram_count()

  def _get_word_count(self):
    x = self.word_cv.fit_transform(self.corpus)
    words = self.word_cv.get_feature_names_out()
    counts = x.todense().sum(axis=0).tolist()[0]
    return {word: count for word, count in zip(words, counts)}

  def _get_bigram_count(self):
    x = self.bigram_cv.fit_transform(self.corpus)
    bigrams = self.bigram_cv.get_feature_names_out()
    counts = x.todense().sum(axis=0).tolist()[0]
    return {word: count for word, count in zip(bigrams, counts)}

  def word_prob(self, word, previous_word):
    """
    MLE : bigram_count / all_bigrams_cointaining_previous_word
    all_bigrams_cointaining_previous_word = previous_word_count

    if laplace_smoothing:
      bigram count += 1
      previous_word_count += vocab size
    """
    word, previous_word = word.lower(), previous_word.lower()
    bigram = previous_word + ' ' + word
    previous_word_count = self.word_count.get(previous_word, 0)
    bigram_count = self.bigram_count.get(bigram, 0)
    if self.laplace_smoothing:
      previous_word_count += self.vocab_size - 1
      bigram_count += 1
    return bigram_count / previous_word_count
  
  def _get_next_word_probs(self, previous_word):
    return {word: self.word_prob(word, previous_word) for word in self.words}

  def next_most_suitable_word(self, sentence):
    """
    It gets the most suitable word after a sentence.
    In case of two values with the same prob, it gets
    one of them.
    """
    previous_word = sentence.split()[-1]
    next_word_probs = self._get_next_word_probs(previous_word)
    return sorted(
        next_word_probs.items(), key=lambda item: item[1], reverse=True)[0]

  def sentence_prob(self, sentence):
    """
    Prob as the composition of independent events approximation.
    p(w_i|w_{1...(i-1)}) ~= p(w_i|w_{1-i})
    then
    p(w_0, w_1, ..., w_i) = p(w_0) * p(w_1|w_0) * ··· * p(w_i|p(w_(i-1))
    """
    sentence = sentence.lower().split()
    pairs = sliding_window_view(sentence, 2)
    probs = [self.word_prob(word, prev_word) for prev_word, word in pairs]
    return np.prod(probs)
  
  def amount_of_information(self, sentence):
    prob = self.sentence_prob(sentence)
    return -np.log(prob), prob

  def perplexity(self, sentence):
    n = len(self.tokenizer(sentence))
    sentence_prob = self.sentence_prob(sentence)
    return np.power(sentence_prob, 1/n)

## Corpus

A continuación definimos el conjunto de entrenamiento dado para la práctica.

In [ ]:
corpus = [
  "<s> I am John </s>",
  "<s> John I am </s>",
  "<s> John I like </s>",
  "<s> John I do like </s>",
  "<s> do I like John </s>"
]

## Modelo sin suavizado

En este apartado vamos a ajustar un modelo con el corpus proporcionado, y vamos a resolver ls distintas secciones.

In [ ]:
bm = BigramModel(corpus)

### Pruebas

Una vez ajustado, vamos a comprobar que el procesado del corpus de prueba ha funcionado correctamente, revisando los conteos de las distintas palabras y los bigramas de los textos:

In [ ]:
bm._get_word_count()

In [ ]:
bm._get_bigram_count()

Antes de empezar con los ejercicios, vamos a hacer un par de cálculos sencillos de probabilidades para comprobar que las probabilidades se calculan adecuadamente.

In [ ]:
bm.word_prob('I', '<s>')

In [ ]:
bm.word_prob('I', 'John')

In [ ]:
bm.sentence_prob("I am John")

### Ejercicios

Vamos a resolver los ejercicios relativos al modelo sin suavizado.

#### 1. ¿Cuál será la palabra siguiente más probable predicha por el modelo para las siguientes secuencias de palabras?
* (1) `<s> John ...`
* (2) `<s> John I do ...`
* (3) `<s> John I am John ...`
* (4) `<s> do I like ...:`

In [ ]:
sentences = [
  "<s> John",
  "<s> John I do",
  "<s> John I am John",
  "<s> do I like"
]

In [ ]:
for sentence in sentences:
  word, prob = bm.next_most_suitable_word(sentence)
  print(f"Sentence: {sentence} ... \t Next word: {word}. \t Probability: {prob}")

#### 2. ¿Cuál de las siguientes frases tiene menor cantidad de información según la teoría de Shannon?
* (5) `<s> John I do I like </s>`
* (6) `<s> John I am </s>`
* (7) `<s> I do like John I am </s>`

In [ ]:
sentences = [
  "<s> John I do I like </s>",
  "<s> John I am </s>",
  "<s> I do like John I am </s>"
]

In [ ]:
for sentence in sentences:
  info, prob = bm.amount_of_information(sentence)
  print(f"Sentences: {sentence}. \t Probability: {prob}. \t Amount of information: {info}")

La frase que tiene asociada una menor cantidad de información es la segunda, que se corresponde con la de mayor probabilidad.

#### Considerando de nuevo el mismo corpus de entrenamiento y el mismo modelo de bigramas, calcule la perplejidad de la siguiente frase:
* `<s> I do like John`

In [ ]:
sentence = "<s> I do like John"

In [ ]:
bm.perplexity(sentence)

## Modelo con suavizado

Tomando de nuevo los mismos datos de entrenamiento, pero esta vez utilizando
un modelo de bigramas con suavizado de Laplace:

In [ ]:
bm_smoothing = BigramModel(corpus, laplace_smoothing=True)

#### Obtenga las probabilidades que asignaría este modelo de bigramas para:
* `P(do|<s>)`
* `P(do|John)`
* `P(John|<s>)`
* `P(John|do)`
* `P(I|John)`
* `P(I|do)`
* `P(like|I)`

**Nota**: para cada palabra wn-1, contamos un bigrama adicional para cada posible continuación wn. En consecuencia, habrá que tener en cuenta, tanto las palabras, como el símbolo de fin de frase (</s>).

Respecto a la nota, estamos tratanto `</s>` como un componente más del vocabulario, por lo que al sumar `V` al numerador, ya se tiene en cuenta. Sin embargo, dado que `w_n` nunca será `<s>` cuando se disponga `w_(n-1)`, en nuestro cado debemos añadir `V - 1` al denominador.

In [ ]:
bm_smoothing.words

In [ ]:
pairs = [
  ("do|<s>"),
  ("do|John"),
  ("John|<s>"),
  ("John|do"),
  ("I|John"),
  ("I|do"),
  ("like|I")
]

In [ ]:
for pair in pairs:
  word, prev_word = pair.split('|')
  prob = bm.word_prob(word, prev_word)
  prob_smoothing = bm_smoothing.word_prob(word, prev_word)
  print("Bigram:", prev_word, word)
  print(f"Without regularization: P({pair}) = {prob}")
  print(f"With regularization: P({pair}) = {prob_smoothing} \n")

#### Calcule las probabilidades de las siguientes secuencias de palabras según
este modelo de lenguaje:
* (8) `<s> do John I like`
* (9) `<s> John do I like`

In [ ]:
sentences = [
    "<s> do John I like",
    "<s> John do I like"
]

In [ ]:
for sentence in sentences:
  prob = bm.sentence_prob(sentence)
  prob_smoothing = bm_smoothing.sentence_prob(sentence)
  print("Sentence:", sentence)
  print(f"Without regularization: P({sentence}) = {prob}")
  print(f"With regularization: P({sentence}) = {prob_smoothing} \n")

Para ambos, la probabilidad sin regularización es 0 porque ambos contienen un bigrama de probabilidad cero dado el corpus de entrenamiento. 

In [ ]:
for sentence in sentences:
  print('\n Sentence', sentence)
  sentence = bm.tokenizer(sentence.lower())
  pairs = sliding_window_view(sentence, 2)
  probs = [bm.word_prob(word, prev_word) for prev_word, word in pairs]
  for pair, prob in zip(pairs, probs):
    print("  ", pair, prob)

Que ambos tengan la misma probabilidad tras al aplicar suavizado es simplemente casualidad, como podemos observar a continuación:

In [ ]:
for sentence in sentences:
  print('\n Sentence', sentence)
  sentence = bm_smoothing.tokenizer(sentence.lower())
  pairs = sliding_window_view(sentence, 2)
  probs = [bm_smoothing.word_prob(word, prev_word) for prev_word, word in pairs]
  for pair, prob in zip(pairs, probs):
    print("  ", pair, prob)